# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI(
  api_key=api_key,
  base_url="https://openrouter.ai/api/v1"
)

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.co

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.com/2026/02/17/ai-coder-vibe-coder-to-agentic-engineer/
https://edwarddonner.

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'resume page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'capabilities page', 'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'portfolio project',
   'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio project', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [ ]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [ ]:
select_relevant_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [9]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [10]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
GGML and llama.cpp join Hugging Face 🔥
Try HuggingChat Omni – Chat with AI 💬
Get started with Inference in seconds 🚀
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
Qwen/Qwen3.5-35B-A3B
Updated
1 day ago
•
158k
•
474
Qwen/Qwen3.5-397B-A17B
Updated
3 days ago
•
602k
•
1.08k
Qwen/Qwen3.5-27B
Updated
1 day ago
•
41.1k
•
322
Qwen/Qwen3.5-122B-A10B
Updated
1 day ago
•
11k
•
291
zai-org/GLM-5
Updated
13 days ago
•
183k
•
1.56k
Browse 2M+ models
Spaces
Running
on
Zero
Featured
1.68k
Qwen Image Multiple Angles 3D Camera
🎥
1.68k
Change the camera angle of a photo with AI
Running
on
Zero
MCP
949
Wan2.2 14B Preview
🐌
949
generate a video from an image with a text prompt
Running
on
Ze

In [11]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [12]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [13]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGGML and llama.cpp join Hugging Face 🔥\nTry HuggingChat Omni – Chat with AI 💬\nGet started with Inference in seconds 🚀\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3.5-35B-A3B\nUpdated\n1 day ago\n•\n158k\n•\n474\nQwen/Qwen3.5-397B-A17B\nUpdated\n3 days ago\n•\n602k\n•\n1.08k\nQwen/Qwen3.5-27B\nUpdated\n1 day ago\n•\n41.1k\n•\n323\nQwen/Qwen3.5-122B-A10B\nUpdated\n1 day ago\n•\n11k\n•\n291\nzai-org/GLM-5\nUpdated\n13 da

In [14]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [15]:
create_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is a vibrant AI community dedicated to building the future of machine learning. It serves as a global collaboration platform where machine learning engineers, scientists, and enthusiasts come together to innovate, share, and experiment with AI models and datasets. The Hugging Face Hub acts as a central marketplace and repository where users can host and explore over 2 million models and half a million datasets spanning multiple modalities such as text, image, video, audio, and even 3D.

Their mission is to empower the next generation of AI builders by fostering an open, ethical, and collaborative environment for AI development. Hugging Face supports these goals with a fast-growing, engaged community and some of the most widely used open-source ML libraries in the world.

---

## What Hugging Face Offers

- **Models Repository:** Access and contribute to a vast collection of 2 million+ pretrained machine learning models for diverse tasks, updated regularly with community contributions.
  
- **Datasets Collection:** Browse over 500,000 public datasets to support a wide range of AI research and applications.
  
- **Spaces:** Deploy and share AI applications and demos easily with community collaboration.
  
- **Community & Collaboration:** Join a thriving global AI community to learn, collaborate, and push the boundaries of open-source AI together.
  
- **Open-Source Stack:** Utilize Hugging Face's robust open-source tools to accelerate your machine learning projects — covering everything from research to deployment.
  
- **Enterprise Solutions:** Professional support and services for enterprises looking to leverage machine learning applications securely and at scale.

---

## Company Culture & Community

Hugging Face champions:

- **Openness:** Encouraging transparency and accessibility in AI research through open-source sharing.
  
- **Collaboration:** Building a supportive ecosystem where developers and researchers co-create and improve AI tools collectively.
  
- **Innovation:** Continuously pushing state-of-the-art developments in machine learning to make AI usable and ethical.
  
- **Learning & Growth:** Providing a platform for individuals to build their machine learning profiles and portfolios by sharing projects and contributions.

---

## Careers and Opportunities

Hugging Face offers exciting career opportunities for talented individuals passionate about advancing AI. Working at Hugging Face means being part of a community-driven company committed to:

- Developing cutting-edge AI technologies.
- Fostering an inclusive workplace.
- Supporting personal and professional growth through collaboration with world-class experts.

Whether you're an engineer, researcher, or community manager, Hugging Face invites you to contribute to shaping the future of machine learning.

---

## Join Hugging Face Today

Explore, create, and collaborate on the home of machine learning. Whether you're developing new models, curating datasets, or deploying AI applications, Hugging Face provides the tools and community to accelerate your journey.

- Visit: [https://huggingface.co](https://huggingface.co)
- Browse models & datasets: 2M+ models, 500k+ datasets available
- Engage with the community and share your work
- Try HuggingChat Omni for conversational AI and more!

---

*Hugging Face – The AI community building the future.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [16]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [17]:
stream_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is a leading platform and vibrant AI community committed to building the future of machine learning. It serves as a collaboration hub where developers, researchers, and enterprises from around the world can share, explore, and innovate on open-source ML models, datasets, and applications.

At the heart of Hugging Face is the commitment to fostering an open and ethical AI future, empowering the next generation of machine learning engineers, scientists, and end users to learn, collaborate, and share their work seamlessly.

---

## Platform Highlights

- **Models:** Access and browse over 2 million open-source AI models spanning text, image, video, audio, and even 3D modalities.
- **Datasets:** Discover from a vast library of 500,000+ datasets to train and improve machine learning models.
- **Spaces:** Deploy, share, and explore interactive AI applications built by the community.
- **Community:** Join a fast-growing network of ML practitioners sharing insights, tools, and advancements.
- **Open Source Stack:** Utilize a comprehensive set of open-source tools developed and maintained by Hugging Face to accelerate AI development.
- **Portfolio Building:** Showcase your ML projects and build your professional portfolio on a respected platform.

---

## Customers and Enterprise Solutions

Hugging Face offers tailored solutions for teams and enterprises, combining the power of its platform with enhanced security and management features:

- **Team and Enterprise Plans:** Subscription-based options starting at $20/user/month with flexible contract options for larger organizations.
- **Enterprise Features Include:**
  - Advanced security with Single Sign-On (SSO), audit logs, and granular access controls.
  - Region-specific data repository selection and private storage options.
  - Analytics dashboards to monitor usage and optimize resource allocation.
  - Scalable compute resources including ZeroGPU with quota boosts.
  - Dedicated support and collaboration tools designed for organizational efficiency.

These features enable enterprises to scale AI development securely and efficiently while maintaining governance and control over their AI assets.

---

## Company Culture

- **Collaboration-Driven:** Hugging Face believes in the power of community collaboration to push the boundaries of AI.
- **Open and Ethical AI:** Dedicated to building AI that is transparent, fair, and benefits everyone.
- **Innovation at the Edge:** Supported by a talented science team exploring cutting-edge technologies and newest advancements.
- **Inclusive Environment:** Welcomes diverse talents passionate about making AI accessible and impactful.

---

## Careers at Hugging Face

Hugging Face is continuously growing and seeks passionate individuals who want to contribute to the AI revolution. Opportunities span software engineering, research science, data engineering, product management, and more.

Working at Hugging Face means joining a global, open-source-driven organization committed to innovation, ethical AI, and community impact. Employees enjoy a culture of learning, collaboration, and technical excellence.

---

## Get Involved

- **Explore the Hub:** Browse millions of models and datasets at https://huggingface.co
- **Join the Community:** Participate in forums, Discord, GitHub repositories, and events.
- **Start Building:** Use Hugging Face tools to jumpstart your ML projects with inference APIs and Spaces.
- **Advance Your Career:** Explore current openings and join the team crafting the future of AI.

---

## Contact and Social Links

- Website: https://huggingface.co
- GitHub: https://github.com/huggingface
- Twitter: https://twitter.com/huggingface
- LinkedIn: https://linkedin.com/company/huggingface
- Discord: Accessible from the main website

---

**Hugging Face** – powering the world’s AI community with openness, innovation, and collaboration. Join us and help build the future of machine learning.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>